In [1]:
# 04/13/2020
# Aline Jaimes
# Source: Open Weather Map
# URL:https://home.openweathermap.org/history_bulks/new
# 
# A json file that contains historical weather data for 5 ENGIE/SoCORE sites from 1978
#  - Connexus Anoka, MN	    45.234472	-93.480455
#  - Connexus Athens, MN	45.460039	-93.265761
#  - Xcel Gopher, MN	    45.398562	-92.679157
#  - Xcel Richmond, MN	    45.458606	-94.513847
#  - Goose Lake, MN	        43.665767	-93.29898

# Preprocessing
# 1) Convert Raw Data to DataFrame
# [1809360 rows x 12 columns]
# 2) Plot data
# 3) Send dataframe to a data table in the Sandbox

# Analysis
#   Background. Moisture affects irradiance sensor and affects data quality
# It is worse when snow covers the panel because it is basically
# moisture in transition period (Dew/Fog).
#   Goal. Analyze moisture in the air.
# From those parameters temperature/ humidity/pressure/ 
# Determine the probable status or dew point.
# Come up with some opinion about the probabily 
# that the moisture in the air is condensed(fog, mist, dew)


In [4]:
# Declare Dependencies
import matplotlib as plt
import mysql.connector
import seaborn as sns
import pandas as pd
import numpy as np
import sshtunnel
import requests
import logging
import pyowm
import time
import json
import os

%matplotlib inline

In [6]:
# Read json data into a dataframe
query_url='http://history.openweathermap.org/storage/a8273f596f2d4a7900fed4dc6554d809.json'
#JSON_string = requests.get(url)
weather_response = requests.get(query_url)
r= weather_response.json()
#r = requests.get(url=link)
dict = r.json()
df=pd.DataFrame(dict)

AttributeError: 'list' object has no attribute 'json'

In [2]:
weather_df = df.DataFrame(columns=['value'])

NameError: name 'df' is not defined

In [ ]:
 def owm_download_pandas(lat, lon):
    weather_df = pd.DataFrame(columns=['value'])
    weather_df.index.name = 'signal'
    location = owm.weather_at_coords(lat, lon)
    weather = location.get_weather()
    #weather_json = json.loads(weather.to_JSON())
    weather_json = dict.loads(weather.to_JSON())

    weather_df.loc['reference_time'] = time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(weather_json['reference_time']))
    weather_df.loc['sunrise_time'] = time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(weather_json['sunrise_time']))
    weather_df.loc['sunset_time'] = time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(weather_json['sunset_time']))
    weather_df.loc['temperature'] = weather.get_temperature('celsius')['temp']
    weather_df.loc['wind_speed'] = weather.get_wind('meters_sec')['speed']
    try:
        weather_df.loc['wind_direction'] = weather_json['wind']['deg']
    except:
        weather_df.loc['wind_direction_deg'] = 0
    weather_df.loc['clouds'] = weather_json['clouds']
    try:
        weather_df.loc['rain'] = weather_json['rain']
    except:
        weather_df.loc['rain'] = 0
    try:
        weather_df.loc['snow'] = weather_json['snow']
    except:
        weather_df.loc['snow'] = 0
    weather_df.loc['humidity_%'] = weather_json['humidity']
    weather_df.loc['pressure'] = weather_json['pressure']['press']
    weather_df.loc['status'] = weather_json['status']
    weather_df.loc['detailed_status'] = weather_json['detailed_status']
    weather_df.loc['weather_code'] = weather_json['weather_code']
    weather_df.loc['visibility_distance'] = weather_json['visibility_distance']
    weather_df.loc['dewpoint'] = weather_json['dewpoint']
    weather_df.loc['humidex'] = weather_json['humidex']
    weather_df.loc['heat_index'] = weather_json['heat_index']
    return weather_df


In [3]:
# Getting the subdictionaries within the dataframe
a=df['main'].apply(pd.Series)
df1=pd.concat([df.drop(['main'], axis=1), a], axis=1)
b=df1['wind'].apply(pd.Series)
df2=pd.concat([df1.drop(['wind'], axis=1), b], axis=1)
c=df2['weather'].apply(pd.Series)
df3=pd.concat([df2.drop(['weather'], axis=1), c], axis=1)
d=df3['clouds'].apply(pd.Series)
df4=pd.concat([df3.drop(['clouds'], axis=1), d], axis=1)

df4.columns

Index([ 'city_name',        'lat',        'lon',         'dt',     'dt_iso',
         'timezone',       'snow',       'rain',       'temp',   'temp_min',
         'temp_max', 'feels_like',   'pressure',   'humidity',      'speed',
              'deg',            0,            1,            2,            3,
              'all'],
      dtype='object')

In [4]:
df4.head()

,city_name,lat,lon,dt,dt_iso,timezone,snow,rain,temp,temp_min,...,feels_like,pressure,humidity,speed,deg,0,1,2,3,all
0,"""Connexus Anoka, MN""",45.234472,-93.480455,283996800,1979-01-01 00:00:00 +0000 UTC,-21600,NaN,NaN,255.33,255.082,...,249.62,1033.0,67.0,2.92,325.0,"{'id': 804, 'main': 'Clouds', 'description': '...",NaN,NaN,NaN,96
1,"""Connexus Anoka, MN""",45.234472,-93.480455,284000400,1979-01-01 01:00:00 +0000 UTC,-21600,NaN,NaN,255.23,254.858,...,249.02,1032.0,68.0,3.63,333.0,"{'id': 804, 'main': 'Clouds', 'description': '...",NaN,NaN,NaN,97
2,"""Connexus Anoka, MN""",45.234472,-93.480455,284004000,1979-01-01 02:00:00 +0000 UTC,-21600,NaN,NaN,255.14,254.710,...,248.81,1032.0,68.0,3.81,335.0,"{'id': 804, 'main': 'Clouds', 'description': '...",NaN,NaN,NaN,100
3,"""Connexus Anoka, MN""",45.234472,-93.480455,284007600,1979-01-01 03:00:00 +0000 UTC,-21600,NaN,NaN,252.71,252.146,...,246.34,1032.0,67.0,3.76,332.0,"{'id': 804, 'main': 'Clouds', 'description': '...",NaN,NaN,NaN,100
4,"""Connexus Anoka, MN""",45.234472,-93.480455,284011200,1979-01-01 04:00:00 +0000 UTC,-21600,NaN,NaN,252.38,251.778,...,245.89,1033.0,67.0,3.92,329.0,"{'id': 804, 'main': 'Clouds', 'description': '...",NaN,NaN,NaN,98


In [7]:
# Filtering data
weather_data = df4.filter(items=['city_name', 'lat', 'lon', 'dt','dt_iso','snow','rain','temp','temp_min','temp_max',
                               'feels_like', 'pressure', 'humidity', 'speed', 'deg', 'all'])

In [13]:
# Saving datatable into a csv file
weather_data.to_csv('OWS_Data.csv',index=False)

In [9]:
weather_data.head(5)

,city_name,lat,lon,dt,dt_iso,snow,rain,temp,temp_min,temp_max,feels_like,pressure,humidity,speed,deg,all
0,"""Connexus Anoka, MN""",45.234472,-93.480455,283996800,1979-01-01 00:00:00 +0000 UTC,NaN,NaN,255.33,255.082,255.646,249.62,1033.0,67.0,2.92,325.0,96
1,"""Connexus Anoka, MN""",45.234472,-93.480455,284000400,1979-01-01 01:00:00 +0000 UTC,NaN,NaN,255.23,254.858,255.719,249.02,1032.0,68.0,3.63,333.0,97
2,"""Connexus Anoka, MN""",45.234472,-93.480455,284004000,1979-01-01 02:00:00 +0000 UTC,NaN,NaN,255.14,254.710,255.698,248.81,1032.0,68.0,3.81,335.0,100
3,"""Connexus Anoka, MN""",45.234472,-93.480455,284007600,1979-01-01 03:00:00 +0000 UTC,NaN,NaN,252.71,252.146,253.446,246.34,1032.0,67.0,3.76,332.0,100
4,"""Connexus Anoka, MN""",45.234472,-93.480455,284011200,1979-01-01 04:00:00 +0000 UTC,NaN,NaN,252.38,251.778,253.186,245.89,1033.0,67.0,3.92,329.0,98


In [11]:
weather_data.columns

Index(['city_name', 'lat', 'lon', 'dt', 'dt_iso', 'snow', 'rain', 'temp',
       'temp_min', 'temp_max', 'feels_like', 'pressure', 'humidity', 'speed',
       'deg', 'all'],
      dtype='object')

In [12]:
# Get data into datbase
# Note: I am unable to connect to the database 
import pymysql.cursors

# Connect to the database
connection = pymysql.connect(host='192.168.2.205',
                             port='3306',
                             user='ajames',
                             password='efbji%*',
                             db='sandbox',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

OperationalError: (2003, "Can't connect to MySQL server on '192.168.2.205' (timed out)")

In [ ]:
# Figures
# Some figures are in PowerBi